<a href="https://colab.research.google.com/github/Chins-10/Data-Analysis-Course/blob/main/covid_19_Interactive_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What is COVID-19?

> Coronaviruses are a large family of viruses that may cause respiratory illnesses in humans ranging from common colds to more severe conditions such as Severe Acute Respiratory Syndrome (SARS) and Middle Eastern Respiratory Syndrome (MERS).1
'Novel coronavirus' is a new, previously unidentified strain of coronavirus. The novel coronavirus involved in the current outbreak has been named SARS-CoV-2 by the World Health Organization (WHO). 3The disease it causes has been named “coronavirus disease 2019” (or “COVID-19”).`

![Coronavirus particle Image](https://www.vtc.edu/wp-content/uploads/2020/07/COVID-banner.jpg)

## Notebook covers:


1. Data loading
2. Data Cleaning and Preparation
3. Visualising N number of worst hit countries using [plotly](https://plotly.com/) scatter plot.
4. Plotting confirmed and death cases for the requested country.
5. Plotting all cases on world map using [Folium](https://python-visualization.github.io/folium/)
6. COVID-19: Progression of spread (Animation)
7. Visualizing covid19 worlwide scenario on [Calender Maps](https://pythonhosted.org/calmap/)


In [1]:
!pip install plotly

In [2]:
# importing libraries

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

In [4]:
# loading data right from the source: John hopkins github repo
death_df = pd.read_csv('time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('cases_country.csv')

In [ ]:
confirmed_df.head().T

,0,1,2,3,4
Province/State,NaN,NaN,NaN,NaN,NaN
Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola
Lat,33.9391,41.1533,28.0339,42.5063,-11.2027
Long,67.71,20.1683,1.6596,1.5218,17.8739
1/22/20,0,0,0,0,0
...,...,...,...,...,...
6/9/22,180864,276583,265920,43224,99761
6/10/22,180864,276638,265925,43224,99761
6/11/22,180864,276690,265925,43224,99761
6/12/22,180864,276731,265927,43224,99761


In [5]:
recovered_df.head().T

,0,1,2,3,4
Province/State,NaN,NaN,NaN,NaN,NaN
Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola
Lat,33.93911,41.1533,28.0339,42.5063,-11.2027
Long,67.709953,20.1683,1.6596,1.5218,17.8739
1/22/20,0,0,0,0,0
...,...,...,...,...,...
6/9/22,0,0,0,0,0
6/10/22,0,0,0,0,0
6/11/22,0,0,0,0,0
6/12/22,0,0,0,0,0


In [6]:
death_df.head().T

,0,1,2,3,4
Province/State,NaN,NaN,NaN,NaN,NaN
Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola
Lat,33.93911,41.1533,28.0339,42.5063,-11.2027
Long,67.709953,20.1683,1.6596,1.5218,17.8739
1/22/20,0,0,0,0,0
...,...,...,...,...,...
6/9/22,7709,3497,6875,153,1900
6/10/22,7709,3497,6875,153,1900
6/11/22,7709,3497,6875,153,1900
6/12/22,7709,3497,6875,153,1900


In [7]:
country_df.head()

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Cases_28_Days,Deaths_28_Days
0,Afghanistan,2022-06-14 13:20:59,33.93911,67.709953,181120,7710,NaN,NaN,465.265139,NaN,NaN,4.256846,4,AFG,1799,19
1,Albania,2022-06-14 13:20:59,41.15330,20.168300,276731,3497,NaN,NaN,9616.060880,NaN,NaN,1.263682,8,ALB,1110,0
2,Algeria,2022-06-14 13:20:59,28.03390,1.659600,265937,6875,NaN,NaN,606.455358,NaN,NaN,2.585199,12,DZA,114,0
3,Andorra,2022-06-14 13:20:59,42.50630,1.521800,43224,153,NaN,NaN,55942.535430,NaN,NaN,0.353970,20,AND,1068,0
4,Angola,2022-06-14 13:20:59,-11.20270,17.873900,99761,1900,NaN,NaN,303.536136,NaN,NaN,1.904552,24,AGO,474,0


In [8]:
# data cleaning

# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})
country_df.head()

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3,cases_28_days,deaths_28_days
0,Afghanistan,2022-06-14 13:20:59,33.93911,67.709953,181120,7710,NaN,NaN,465.265139,NaN,NaN,4.256846,4,AFG,1799,19
1,Albania,2022-06-14 13:20:59,41.15330,20.168300,276731,3497,NaN,NaN,9616.060880,NaN,NaN,1.263682,8,ALB,1110,0
2,Algeria,2022-06-14 13:20:59,28.03390,1.659600,265937,6875,NaN,NaN,606.455358,NaN,NaN,2.585199,12,DZA,114,0
3,Andorra,2022-06-14 13:20:59,42.50630,1.521800,43224,153,NaN,NaN,55942.535430,NaN,NaN,0.353970,20,AND,1068,0
4,Angola,2022-06-14 13:20:59,-11.20270,17.873900,99761,1900,NaN,NaN,303.536136,NaN,NaN,1.904552,24,AGO,474,0


In [9]:
country_df.shape

(199, 16)

In [10]:
country_df.isna().sum()

country                  0
last_update              0
lat                      2
long_                    2
confirmed                0
deaths                   0
recovered              199
active                 199
incident_rate            5
people_tested          199
people_hospitalized    199
mortality_rate           0
uid                      0
iso3                     4
cases_28_days            0
deaths_28_days           0
dtype: int64

In [11]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

In [12]:
# displaying the total stats

display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

# COVID-19 Confirmed/Death/Recovered cases by countries

## Enter number of countries you want the data for

In [13]:
# sorting the values by confirmed descednding order
#country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g

    return df1

def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
#ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [14]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

In [15]:
fig = px.scatter(sorted_country_df.head(10), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
fig.update_layout(
    title=str(10) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
fig.show();


# Slide to check for the worst hit countries

In [16]:
# # plotting the 20 worst hit countries

def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show();

#interact(bubble_chart, n=10)
#ipywLayout = widgets.Layout(border='solid 2px green')
#ipywLayout.display='none'
#widgets.VBox([fig], layout=ipywLayout)

In [17]:


def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]

    df_list = [confirmed_df, death_df]

    fig = go.Figure();

    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)

        else:
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)

        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));

    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,

    );

    fig.update_yaxes(type="linear")
    fig.show();
plot_cases_of_a_country('World')

In [18]:
plot_cases_of_a_country('India')

In [19]:
plot_cases_of_a_country('US')

# Check the details of any country or the World

* Enter the name of your country(in capitalized format(e.g. Italy)) and world for total cases

# 10 worst hit countries - Confirmed cases

In [20]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "confirmed",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"],
    height=500,
    width=800
)

# 10 worst hit countries - Death cases

In [21]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "deaths",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"],
    height=500,
    width=800
)

# Worst hit countries - Recovering cases

In [22]:
  len(sorted_country_df)

199

In [23]:
sorted_country_df.isnull().sum()

country                  0
last_update              0
lat                      2
long_                    2
confirmed                0
deaths                   0
recovered              199
active                 199
incident_rate            5
people_tested          199
people_hospitalized    199
mortality_rate           0
uid                      0
iso3                     4
cases_28_days            0
deaths_28_days           0
dtype: int64

In [24]:
  sorted_country_df.head(10)

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3,cases_28_days,deaths_28_days
184,US,2022-06-14 13:20:59,40.000000,-100.000000,85633278,1011545,NaN,NaN,25991.514889,NaN,NaN,1.181252,840,USA,2955626,9411
80,India,2022-06-14 13:20:59,20.593684,78.962880,43236695,524777,NaN,NaN,3133.083885,NaN,NaN,1.213731,356,IND,111325,517
24,Brazil,2022-06-14 13:20:59,-14.235000,-51.925300,31497038,668180,NaN,NaN,14817.992837,NaN,NaN,2.121406,76,BRA,795138,2964
63,France,2022-06-14 13:20:59,46.227600,2.213700,30057668,149880,NaN,NaN,46065.502410,NaN,NaN,0.498641,250,FRA,565562,1272
67,Germany,2022-06-14 13:20:59,51.165691,10.451526,26915085,139914,NaN,NaN,32367.356101,NaN,NaN,0.519835,276,DEU,1096680,2200
188,United Kingdom,2022-06-14 13:20:59,55.000000,-3.000000,22600145,179941,NaN,NaN,33291.317309,NaN,NaN,0.796194,826,GBR,214942,1878
94,"Korea, South",2022-06-14 13:20:59,35.907757,127.766922,18239056,24390,NaN,NaN,35575.086110,NaN,NaN,0.133724,410,KOR,408627,619
146,Russia,2022-06-14 13:20:59,61.524000,105.318800,18113989,372452,NaN,NaN,12412.413764,NaN,NaN,2.056157,643,RUS,112802,2247
86,Italy,2022-06-14 13:20:59,41.871900,12.567400,17664043,167432,NaN,NaN,29215.198389,NaN,NaN,0.947869,380,ITA,592394,2086
183,Turkey,2022-06-14 13:20:59,38.963700,35.243300,15072747,98965,NaN,NaN,17871.607472,NaN,NaN,0.656582,792,TUR,17171,61


In [25]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "recovered",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"],
    height=500,
    width=800
)